<div class='alert alert-block alert-warning' style="font-size: 20pt; font-family: cursive; color: #08415C"> 
    <b> <center> Web Scraping Data</center> </b>
</div>

<div style="font-size: 13pt; font-family: verdana; color: #08415C"> 
    The dataset has a total of 6 columns.
    <ul>
        <li>name - name of the residential complex</li>
        <li>link - link from the site</li>
        <li>address - address of the residential complex</li>    
        <li>square_price - price per square meter</li>
        <li>Area - the area of the apartment</li>
        <li>price_start_from - inital prices for an apartment</li>
    </ul>
</div>

In [50]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [51]:
url = "https://krisha.kz/complex/search/"

r = requests.get(url)

In [52]:
soup = BeautifulSoup(r.text, 'html')

In [53]:
main_card = soup.find('div', class_='complex-card complex-card--not-empty complex-card--with-result')
main_card

<div class="complex-card complex-card--not-empty complex-card--with-result" data-complex-alias="almaty/ramsevo" data-complex-id="16937355" data-complex-region="Алматы" data-filter-city="2" data-free-call-visibility="" data-has-buttons="1" data-is-free="" data-name="RAMS EVO" data-work-time-from="09:00" data-work-time-to="19:00">
<div class="complex-card__state">
                Строящийся
            </div>
<a class="complex-card__main-block complex-card__main-block--hypothec" href="/complex/show/almaty/ramsevo/" target="_blank">
<div class="complex-card__child-block flats">
<div class="complex-card__info">
<div class="complex-card__header"><picture>
<img alt="RAMS EVO" loading="lazy" src="https://krisha-photos.kcdn.online/07/f0fe5bfaf6daca6c0e8a2c88a54034807d793f/photo-448x280.jpg"/>
</picture>
</div>
</div>
<div class="complex-card__content">
<span class="credit-badge credit-badge--hypothec-full">
<i class="credit-badge__icon fi-home-bordered"></i>
                                   

In [54]:
link = 'https://krisha.kz' + main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").get("href")
link

'https://krisha.kz/complex/show/almaty/ramsevo/'

In [55]:
name = main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('p', class_='complex-card__title').text.strip()
name

'ЖК RAMS EVO'

In [56]:
address = main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('p', class_='complex-card__address').text.strip()
address

'Алматы, Алмалинский р-н, пересечение Ауэзова и Гоголя'

In [57]:
square_price = ''.join(filter(str.isdigit, main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('p', class_='complex-card__square-price').text)) + 'tg'
square_price

'630000²tg'

In [58]:
Area = main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('div', class_='complex-card__price-item complex-card__price-item--result').find('span').text
Area

'1-комн. 41.9 м²'

In [59]:
price_start_from = ''.join(filter(str.isdigit, main_card.find('a', class_="complex-card__main-block complex-card__main-block--hypothec").find('span', class_='complex-flat-price').text))
price_start_from

'35728498'

In [60]:
text = price_start_from
numeric_value = ''.join(filter(str.isdigit, text))
price_start_from = int(numeric_value)
price_start_from

35728498

In [61]:
flats = soup.findAll('div', class_='complex-card complex-card--not-empty complex-card--with-result')
len(flats)

12

In [62]:
flat_dict = {}

for p in range(1, 88):
    
    print('Web scraping page:', p)
    
    url = f"https://krisha.kz/complex/search/ramscity/?state[]=1&state[]=2&page={p}"
    r = requests.get(url)
    sleep(1)
    soup = BeautifulSoup(r.text, 'lxml')
    
    flats = soup.findAll('a', class_="complex-card__main-block complex-card__main-block--hypothec")
    
    

    for flat in flats:
        
        flat_name = flat.find('p', class_='complex-card__title').text.strip()
        flat_dict[flat_name] = {}  # Create a nested dictionary for each flat
        
        try:
            link = 'https://krisha.kz' + str(flat.get("href"))
            flat_dict[flat_name]['link'] = link
        except AttributeError:
            flat_dict[flat_name]['link'] = None
            
        try:
            address_element = flat.find('p', class_='complex-card__address')
            flat_dict[flat_name]['address'] = address_element.text.strip()
        except AttributeError:
            flat_dict[flat_name]['address'] = None

        try:
            square_price_element = flat.find('p', class_='complex-card__square-price')
            square_price = ''.join(filter(str.isdigit, square_price_element.text)) + 'tg'
            flat_dict[flat_name]['square_price'] = square_price
        except AttributeError:
            flat_dict[flat_name]['square_price'] = None

        try:
            area_element = flat.find('div', class_='complex-card__price-item complex-card__price-item--result').find('span')
            flat_dict[flat_name]['Area'] = area_element.text
        except AttributeError:
            flat_dict[flat_name]['Area'] = None

        try:
            price_element = flat.find('span', class_='complex-flat-price')
            price_start_from = ''.join(filter(str.isdigit, price_element.text))
            flat_dict[flat_name]['price_start_from'] = price_start_from
        except AttributeError:
            flat_dict[flat_name]['price_start_from'] = None

    

# Now, the 'data' dictionary contains data for each flat with possible missing values set to None


Web scraping page: 1
Web scraping page: 2
Web scraping page: 3
Web scraping page: 4
Web scraping page: 5
Web scraping page: 6
Web scraping page: 7
Web scraping page: 8
Web scraping page: 9
Web scraping page: 10
Web scraping page: 11
Web scraping page: 12
Web scraping page: 13
Web scraping page: 14
Web scraping page: 15
Web scraping page: 16
Web scraping page: 17
Web scraping page: 18
Web scraping page: 19
Web scraping page: 20
Web scraping page: 21
Web scraping page: 22
Web scraping page: 23
Web scraping page: 24
Web scraping page: 25
Web scraping page: 26
Web scraping page: 27
Web scraping page: 28
Web scraping page: 29
Web scraping page: 30
Web scraping page: 31
Web scraping page: 32
Web scraping page: 33
Web scraping page: 34
Web scraping page: 35
Web scraping page: 36
Web scraping page: 37
Web scraping page: 38
Web scraping page: 39
Web scraping page: 40
Web scraping page: 41
Web scraping page: 42
Web scraping page: 43
Web scraping page: 44
Web scraping page: 45
Web scraping page: 

In [63]:
len(flat_dict)

800

In [64]:
import pandas as pd

df = pd.DataFrame(flat_dict)

df = df.transpose()

pd.set_option('display.max.rows', 834)

df.head()

,link,address,square_price,Area,price_start_from
ЖК RAMS EVO,https://krisha.kz/complex/show/almaty/ramsevo/,"Алматы, Алмалинский р-н, пересечение Ауэзова и...",630000²tg,1-комн. 41.9 м²,35728498
ЖК RAMS CITY,https://krisha.kz/complex/show/almaty/ramscity/,"Алматы, Бостандыкский р-н, ул. Жандосова, 94А",590000²tg,3-комн. 76.4 м²,54014800
ЖК RAMS SAIAHAT,https://krisha.kz/complex/show/almaty/ramssaia...,"Алматы, Жетысуский р-н, пересечении улиц Райым...",595000²tg,1-комн. 37.6 м²,25229600
ЖК Столичный 2,https://krisha.kz/complex/show/nur-sultan/stol...,"Астана, Сарыарка р-н, ул. Абая, 10/2",400000²tg,1-комн. 39.4 м²,16548000
ЖК Горное Солнце,https://krisha.kz/complex/show/gornoe-solnce/,"Алматы, Бостандыкский р-н, Проспект Аль-Фараби...",960000²tg,2-комн. 75.93 м²,72864000


<div class='alert alert-block alert-warning' style="font-size: 20pt; font-family: cursive; color: black"> 
    <b> <center><div class='alert alert-block alert-warning' style="font-size: 20pt; font-family: cursive; color: black"> 
    <b> <center>correction of the price column</center> </b>
</div></center> </b>
</div>

In [65]:
import re 

df['square_price'] = df['square_price'].astype(str)
# Use regular expressions to extract the numeric part
df['square_price'] = df['square_price'].str.extract(r'(\d+)')

new_order = ['Area', 'square_price', 'price_start_from', 'link', 'address']

df = df[new_order]

df.head()

,Area,square_price,price_start_from,link,address
ЖК RAMS EVO,1-комн. 41.9 м²,630000,35728498,https://krisha.kz/complex/show/almaty/ramsevo/,"Алматы, Алмалинский р-н, пересечение Ауэзова и..."
ЖК RAMS CITY,3-комн. 76.4 м²,590000,54014800,https://krisha.kz/complex/show/almaty/ramscity/,"Алматы, Бостандыкский р-н, ул. Жандосова, 94А"
ЖК RAMS SAIAHAT,1-комн. 37.6 м²,595000,25229600,https://krisha.kz/complex/show/almaty/ramssaia...,"Алматы, Жетысуский р-н, пересечении улиц Райым..."
ЖК Столичный 2,1-комн. 39.4 м²,400000,16548000,https://krisha.kz/complex/show/nur-sultan/stol...,"Астана, Сарыарка р-н, ул. Абая, 10/2"
ЖК Горное Солнце,2-комн. 75.93 м²,960000,72864000,https://krisha.kz/complex/show/gornoe-solnce/,"Алматы, Бостандыкский р-н, Проспект Аль-Фараби..."


In [66]:
# dividing the address into 4 columns
df[['City', 'district', 'Street', 'house_number']] = df['address'].str.split(',', 3, expand=True)

C:\Users\Зангар\AppData\Local\Temp\ipykernel_13556\1694347654.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['City', 'district', 'Street', 'house_number']] = df['address'].str.split(',', 3, expand=True)


In [67]:
# Save to CSV
df.to_csv('krishaKZ.csv', index=False)  # Set index=False to exclude the DataFrame index in the CSV file

In [69]:
df = pd.read_csv('KrishaKZ.csv')
df.head()

,Area,square_price,price_start_from,link,address,City,district,Street,house_number
0,1-комн. 41.9 м²,630000.0,35728498.0,https://krisha.kz/complex/show/almaty/ramsevo/,"Алматы, Алмалинский р-н, пересечение Ауэзова и...",Алматы,Алмалинский р-н,пересечение Ауэзова и Гоголя,NaN
1,3-комн. 76.4 м²,590000.0,54014800.0,https://krisha.kz/complex/show/almaty/ramscity/,"Алматы, Бостандыкский р-н, ул. Жандосова, 94А",Алматы,Бостандыкский р-н,ул. Жандосова,94А
2,1-комн. 37.6 м²,595000.0,25229600.0,https://krisha.kz/complex/show/almaty/ramssaia...,"Алматы, Жетысуский р-н, пересечении улиц Райым...",Алматы,Жетысуский р-н,пересечении улиц Райымбека и Суюнбая,NaN
3,1-комн. 39.4 м²,400000.0,16548000.0,https://krisha.kz/complex/show/nur-sultan/stol...,"Астана, Сарыарка р-н, ул. Абая, 10/2",Астана,Сарыарка р-н,ул. Абая,10/2
4,2-комн. 75.93 м²,960000.0,72864000.0,https://krisha.kz/complex/show/gornoe-solnce/,"Алматы, Бостандыкский р-н, Проспект Аль-Фараби...",Алматы,Бостандыкский р-н,Проспект Аль-Фараби,144
